# Team 7 Airline Delays

## Additional Materials
Github Repo:
https://github.com/UCB-w261/su20-project-su20-team7

EDA Notebook:
https://dbc-9bfca75b-725c.cloud.databricks.com/?o=52497824689954#notebook/2542098129693526/command/2542098129693527

##1 Introduction to the Problem
The FAA annually handles over 16 million flights out of 20,000 airports (about 5K public commercial and 15K private) to service 1 billion passengers and 44 trillion tons of freight. This translates to about 44,000 flights operating daily serving about 3 million passengers. The aviation industry supports 10 million jobs and contributes 5.1% to the U.S. GDP. Given the expanse and complexity of operations, delays in airline and airport operations are therefore inevitable. As of 2018, almost 1 in 5 US flights suffered a delay and as of 2019, worldwide costs due to airline delays were estimated at $60 billion, with over a half incurred due to delays in US domestic as well as international flights. Economic costs of delayed flights not only include the direct effects of increased airline costs (due to crew, fuel, aircraft and maintenance) and airport costs (operations, maintenance, supporting service provides concessions, transportation) but also the indirect effects of lost labour productivity for business travellers, an opportunity cost of time for leisure travellers (due to schedule buffer, delayed flights, flight cancellations and missed connections), and changes in consumer spending on travel and tourism goods and services. As of 2019 US aviation reported a fuel wastage of 740 million gallons and CO2  emissions amounting to 7.1 million tons. Clearly this problem has a huge impact in several areas. 

With the FAA forecasting an increase of 65% over the next couple decades in revenue passenger miles, finding solutions to mitigate delays is imperative. A 2012 study suggests that “even for modest reductions in flight delay, the economic benefits are substantial. US net welfare would increase by $17.6 billion for a 10 per cent reduction in flight delay and by $38.5 billion for a 30 per cent reduction. ”

[Slide01-Rad]

In [4]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/delays_overtime.png?token=AFP5XG375TQCOWM6FCPEHS27GVNEQ' >")

In [5]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/flights%20by%20year.png?token=AKFL6ADLX5OBKYO2FDHF4NK7GVP42' >")

Efforts so far to address this problem have taken the shape of expanding aviation infrastructure, improvements to existing air traffic management systems and congestion pricing. The first is costly and depends upon availability of real-estate and funds. OHare added a 5th runway recently and is investing 8.5 billion for terminal and runway expansions, 10 additional gates, 2 satellite terminals, and 60,000 jobs.  The second improves airport capacity by better air traffic management, while the third works on reducing demand during peak periods and its effectiveness is based on the willingness of travellers to consider off-peak pricing. 

Computationally speaking, researchers have taken several approaches to tackle this problem. Many involve statistical analysis, probabilistic modeling, network modeling,  and operations research techniques with most solutions focused on reducing aggregate delays at airports by forecasting the same or trying to understand the propagation of delays after the first delay has already occurred. 

While this problem has been computationally intensive and complex to solve historically, in recent years, improvements in big data technology and machine learning have led to new research that involves developing scalable algorithms that can predict delays at an individual flight level with the help of openly sourced data. While no such solutions are currently “operationalized”, these are imagined to work alongside existing Air Traffic Management Systems to improve air space management, and help airlines, airports and passengers respond to delays and mitigate losses proactively.

Our project showcases the design of one such approach. Flight Performance Data from the FAA (Federal Aviation Authority) and meteorological information from NOAA (National Oceanic and Atmospheric Administration) are considered to make a prediction of whether a flight would incur a delay of greater than 15 minutes or not. This process is expected to be run 2 hrs prior to flight departures at any point of time (giving airlines and airports time to regroup and passengers proactively upon a delay), and deliver a prediction accuracy of about 85% (which is the current industry best).

[Slide2-Rad]

#### Goal: To develop an algorithm that would predict whether a flight is going to be delayed by >15 min, given airlines and weather data, 2hrs prior to flight departures, with a target weighted precision score of at least 85% and a target prediction time of 30 minutes.

###Open Datasets

The passenger flights on-time performance data primarily consisted of origin-destination, carrier, departure/arrival performance, delays, causes of delay,  flight summaries, gate info and diverted flight information (a total of 109 fields ). 

The weather data consisted of primary weather measurements such as temperature, precipitation, wind, humidity etc. taken hourly at various weather stations world over, along with several other data related to the quality and frequency of the primary measurements (a total of 177 fields). The existence of these data in parquet format in the Databricks file system enabled pushdown querying and efficient joins.

The airports data and the holidays list were scraped from the web.

In [10]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/data%20specifications.png?token=AKFL6AHQKKUWMLIW7YQELBK7GVRBC' >")

### Initial EDA
As a first step, we attempted to understand a few aspects of this data. 

First, the distribution of delays by various characteristics and second, distribution of delays by delay type, and more specifically, the contribution of weather to these delays. 

The former would help us understand what elements of the airlines’ data could potentially be used to signal a delay, while the latter would help us determine if there are any particular weather related fields that could help us improve the accuracy of our forecasts more so than others.

[Slide3-Rad]

####Delay Analysis
 - by delay duration
 - by type of delay
 - weather's contribution

In [14]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/delays%20by%20duration.png?token=AKFL6AGY3VB6GMRQIG2DWK27GVSXK' >")

#### Delay Types

[Slide4-Rad]

In [17]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/delays%20by%20delay%20type.png?token=AKFL6AB2HFX6FFB2KVITTF27GVR7E' >")

####Weather's Contribution
Our initial exploratory analysis and further research around delays revealed that while flight delays vary by carrier, day of week , month of year and distance, the variation of delays attributed due to weather were not completely captured by the WEATHER_DELAY field as we first imagined. 

It was useful to learn that the  WEATHER_DELAY field only captures about 6% of the total weather related delays while the remaining are actually included in the NAS_DELAY  and LATE_AIRCRAFT_DELAY. 

Prior research in the area and information by BTS (Bureau of Transportation Statistics) confirmed that weather delays may constitute as much as 40% of the total delays and as much as 20% of delays are caused by Late Arriving aircrafts.

This knowledge helped us establish perspective on a few things:
 - to avoid the potential pitfall of eliminating records that were in some instances not coded as delayed due to weather for training
 - to place undue importance on weather measurements as being the dominating  predictor of delay. 
 - to experiment and compare the accuracy of different models, one trained on features that include weather related data Vs another trained on features that include just flight information.
 - while weather data is important, since joins between massive tables could be expensive, we could consider bringing in a subset of columns required for the success of our model without negatively impacting the performance of our data pipeline.

In [19]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/weather%20share%20of%20delays.png?token=AKFL6ADY3HXQMKW5JDLR4EC7GVTP6' >")

[Slide9-Sirak]

### Data Distributions

Images from our EDA Notebook, linked: https://dbc-9bfca75b-725c.cloud.databricks.com/?o=52497824689954#notebook/2542098129693526/command/2542098129693527

#### Delays by Airport

In [23]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/AirportTallies.png?token=AFP5XG347UKOP62XQUNTCZS7GWBUK' >")

In [24]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/delays_USmap.png?token=AFP5XGZG72HRBYTCABVWA6C7GWBVG' width=800 height=550>")

#### Delays over Time

In [26]:
# Hypothesis, long holiday's are a good indicator for delays.
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/long_weekends.png?token=AFP5XG2V4LPKZAL36LLHC6K7GWBV2' >")

In [27]:
# Tip! The last Thurs of Nov is best day of the year to avoid delays
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/delays_overtime.png?token=AFP5XGYPI2ROEHSNU5YPBJK7GWBVO' >")

#### Weather Distributions

The following are taken from a 1% sample of the training years 2015-2017

In [29]:
# We decided to keep values entered as "Missing" due to their sheer number, 
# and added a one-hot encoding to keep track of these missing entries
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/temperatures.png?token=AFP5XG2Q6UPUBDZARLLLRPS7GWBXE' >")

In [30]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/dewpoints.png?token=AFP5XGZZ3H3DHELZIO5MUC27GWBVU' >")

In [31]:
# Wind angles reporte as missing where windspeed was reported were imputed to zero as well, and not recorded as missing
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/windspeeds.png?token=AFP5XG275XE2SYFFCITSYBS7GWBXS' >")

In [32]:
# Scatter plot with Airports columns
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/scatter_depdel_airlines.png?token=AFP5XG7WTRFBPN2TPOERA3S7GWEYI'  width=800 height=800>")

In [33]:
# Scatter with weather columns
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/scatter_depdel_weather.png?token=AFP5XGZFUMDF7ZOBDM3W3B27GWEYA'  width=900 height=900>")

## Imports

In [36]:
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType, TimestampType
from pyspark.sql import SQLContext
from pyspark.sql.functions import trim
import plotly.express as px
import urllib.request
from pyspark.sql import Window
from pyspark.sql.functions import hour
from pyspark.sql.functions import udf, concat, lit, col
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


sqlContext = SQLContext(sc)


/databricks/spark/python/pyspark/sql/context.py:77: DeprecationWarning:

Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

## Utilities

In [38]:
# FUNCTION TO CHECK IF FILE EXISTS
# From https://forums.databricks.com/questions/20129/how-to-check-file-exists-in-databricks.html
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    return True
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise
      
# FUNCTION TO COUNT NULLS/NANS
# MAX: added some type safety
def check_nulls_nans(df):
    null_counts_df = df.select([f.count(f.when(f.isnan(c.name) | f.isnull(c.name), c.name)).alias(c.name) for c in df.schema.fields if not isinstance(c.dataType, TimestampType) ])
    return null_counts_df

## Environment

In [40]:
# ENVIRONMENTAL VARIABLES SETUP
# Run validations: True | False
VALIDATE=False
# Verbose: True | False
VERBOSE=False
# Don't use persisted data on disk, instead recompute everything: True | False
FORCE_COMPUTE_ALL=False
# Persist everything to disk: True | False
PERSIST_ALL=False
# Persist models to disk: True | False
PERSIST_MODELS = False
# Datasets: 
# 'Toy': Initial dev data
# '2015': All of 2015
# '2015_2017': 2015-2017
DATASETS='2015_2019'

## User Environment

In [42]:
# Enter "MR7" to load models from disk
user_initials = "MR7"
if user_initials == "MR7":
  PERSIST_MODELS = False # ensuring that the golden models stored under MR7 folder are not overwritten!!

[Slide5-Max]

### Data Pipeline 
For the purpose of organizing our data engineering, we adopted a model like the one Databricks documents, using a metaphor of bronze, silver and gold datasets. We use the terms in this way:

**Bronze** - thin facade over the raw data sources to make them usable in spark.

**Silver** - processed, regularized, joined datasets

**Gold** finished data ready for ML modeling

This helps divide tasks and responsibilities. EDA and data regularization (normalization, correction, imputation) happen in Silver. Feature engineering also happens in Silver, although high-level feature engineering such as feature selection and PCA happen in Gold. We found it useful to introduce sub-groups to Silver: "clean" tables have data regularization. Pure silver tables may be the result of joins and may be enriched with derived features.

## Bronze Datasets 
Bronze datasets will be named as follows:
* airports_raw
* weather_raw
* airlines_raw
* holidays_raw

In this first section, we will establish RDDs and SQL tables with those names. 

There is no need to persist the bronze tables since they are just views of raw data.

### airports_raw

Source: https://openflights.org/data.html

In [47]:
# FUNCTION TO SCRAPE AIRPORTS DATA
def init_airports_raw():
  # Note: the airports.dat data is very small, so we will always load it in full
  source_url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
  data_file_name = 'dbfs:/team7/data/airports.dat'
  if not file_exists(data_file_name):
    print(f"Importing the data from {source_url}")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", "/tmp/airports.dat") 
    dbutils.fs.mv("file:/tmp/airports.dat", data_file_name)
  else:
    print(f"Skipping import: data already exists at {data_file_name}")
  # display(dbutils.fs.ls("/data/airports.dat"))
  airports_schema = StructType([
    StructField("Airport_ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("IATA", StringType(), True),
    StructField("ICAO", StringType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True),
    StructField("Altitude", IntegerType(), True),
    StructField("Timezone", DoubleType(), True),
    StructField("DST", StringType(), True),
    StructField("Tz", StringType(), True),
    StructField("Type", StringType(), True),
    StructField("Source", StringType(), True)
  ])
  df = spark.read.format("csv") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("sep", ",") \
    .schema(airports_schema) \
    .load(data_file_name)
  return df

In [48]:
# IMPORT AIRPORTS DATA & CREATE A VIEW
airports_raw = init_airports_raw()
airports_raw.createOrReplaceTempView("airports_raw")
print(f"airports_raw: {airports_raw.count()} rows")

Skipping import: data already exists at dbfs:/team7/data/airports.dat
airports_raw: 7698 rows

In [49]:
# PRINT SCHEMA
if VERBOSE:
  airports_raw.printSchema()

In [50]:
# DISPLAY DATA
if VERBOSE:
  display(airports_raw)

###Long Holidays 
(Via https://docs.google.com/spreadsheets/d/1zv-Sydffde6luh_lVTcFidjpjsK7ojjXRciAUelbQr4/edit#gid=749082549)

In [52]:
import requests
from pyspark.sql.functions import lit

# FUNCTION TO SCRAPE Long-Holidays DATA
def init_holidays_raw():
    # Grab the Google Sheets doc and save it to file
    source_url = 'https://docs.google.com/spreadsheet/ccc?key=1zv-Sydffde6luh_lVTcFidjpjsK7ojjXRciAUelbQr4&output=csv'
    data_file_name = 'dbfs:/team7/data/holidays.csv'
    if not file_exists(data_file_name):
        print(f"Importing the data from {source_url}")
        response = requests.get(source_url) 
        assert response.status_code == 200, 'Wrong status code'
        dbutils.fs.put(data_file_name, str(response.content, 'utf-8'), True)
    else:
        print(f"Skipping import: data already exists at {data_file_name}")
        
    holidays_schema = StructType([
        StructField("Year", IntegerType(), True),
        StructField("Date", StringType(), True),
        StructField("Weekday", StringType(), True),
        StructField("Name", StringType(), False),
        StructField("Type", StringType(), False),
        ])
    
    df = spark.read.format("csv") \
        .option("inferSchema", "false") \
        .option("header", "true") \
        .option("sep", ",") \
        .schema(holidays_schema) \
        .load(data_file_name)
    
    date_split = f.split(df.Date, '/')

    return (df
            .withColumn("holiday", lit(1))
           )

In [53]:
# SCRAPE Long Holiday DATA
holidays_raw = init_holidays_raw()
holidays_raw.createOrReplaceTempView("holidays_raw")

Skipping import: data already exists at dbfs:/team7/data/holidays.csv

In [54]:
if VALIDATE:
  display(holidays_raw)

### weather_raw
Source: https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

In [56]:
# FUNCTION TO IMPORT WEATHER DATA
def init_weather_raw():
  print("Loading weather")
  if DATASETS == 'Toy':
    df = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet")

    # extract weather data for the 1st quarter of 2015
    df = df.filter(f.col('date').between("2015-01-01", "2015-04-01"))
  elif DATASETS == '2015':
    df = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet")
  elif DATASETS == '2015_2017':
    df = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather201[5-7]a.parquet")
  elif DATASETS == '2015_2019':
    df = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather201[5-9]a.parquet")
  else:
    raise RuntimeException(f"Bad value for {DATASETS}")
  return df


In [57]:
# IMPORT WEATHER DATA & CREATE A VIEW
weather_raw = init_weather_raw()
weather_raw.createOrReplaceTempView("weather_raw")
print(f"weather_raw: {weather_raw.count()} rows")

Loading weather
weather_raw: 626994336 rows

In [58]:
# PRINT SCHEMA
if VERBOSE:
  weather_raw.printSchema()

In [59]:
# DISPLAY DATA
if VERBOSE:
  display(weather_raw)

In [60]:
# verify quarterly counts
query = f"""
    WITH quarters AS (
    select date_trunc("QUARTER", DATE) as quarter, 
           count(*) as count
    from weather_raw
    group by quarter
    )
    select date_format(quarter, "yyyy") as Year,
           date_format(quarter, "q") as Quarter, 
           count from quarters
    order by Year, Quarter
"""
if VERBOSE:
  df = spark.sql(query)
  display(df)

### airlines_raw
Source:

In [62]:
# FUNCTION TO IMPORT AIRLINES DATA
def init_airlines_raw():
  if DATASETS == 'Toy':
    airlines_raw = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/*.parquet")
  elif DATASETS == '2015':
    airlines_raw = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2015.parquet")
  elif DATASETS == '2015_2017':
    airlines_raw = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201[5-7].parquet")
  elif DATASETS == '2015_2019':
    airlines_raw = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201[5-9].parquet")
  else:
    raise RuntimeException(f"Bad value for {DATASETS}")
  return airlines_raw


In [63]:
# IMPORT AIRLINES DATA & CREATE A VIEW
airlines_raw = init_airlines_raw()
airlines_raw.createOrReplaceTempView("airlines_raw")
if VALIDATE:
  print(f"airlines_raw: {airlines_raw.count()} rows")

In [64]:
# PRINT SCHEMA
if VERBOSE:
  airlines_raw.printSchema()

In [65]:
# DISPLAY DATA
if VERBOSE:
  display(airlines_raw)


In [66]:
# validate quarterly counts
query = f"""
    WITH quarters AS (
      select YEAR, QUARTER, count(*) as count
      from airlines_raw
      group by YEAR, QUARTER
    )
    select YEAR as Year,
           QUARTER as Quarter, 
           count from quarters
    order by Year, Quarter
"""
if VALIDATE:
  print(query)
  df = spark.sql(query)
  display(df)


In [67]:
# join with airlines and holidays
def join_longHolidays_toAirlines(airlinesDF, holidaysDF):
    df = (holidaysDF.select("holiday","Date")
                  .withColumnRenamed("Date","FL_Date")
          .join(airlinesDF, on="FL_Date", how="right") )
    
    return df.fillna({'holiday':0})
  
airlines_raw = join_longHolidays_toAirlines(airlines_raw,holidays_raw)

In [68]:
if VERBOSE :
  display(airlines_raw)

## Silver Datasets
These have derived, cleaned and regularized data.
* airports_clean
* airlines_clean
* airlines_silver (airlines joined with airports)
* weather_clean
* weather_silver (weather joined with airports)
* silver_df (airlines-airports-weather)

####Data Cleaning

[NS5]Once we figured these critical aspects we progressed with data import, cleansing, transforming, and joining our airlines and weather data.

As a part of data import, we imported the following raw data and stored it as our Bronze Data sets:
Airports.data
Holidays data
Airlines data
Weather data

As a part of data cleansing, we:
eliminated duplicates
imputed nulls to zeroes wherever applicable
handled invalid time zones by setting them to Eastern
Assessed the impact of invalid airport codes and airport codes that could not be mapped to weather stations. There were two flight origins (local/municipal airports) in our training data that could not be mapped to weather stations. These accounted to <1% of total flights. Given the characteristics, we decided to drop these from our training set downstream.

As a part of data transformations, we:
converted the arrival and departure times of flights to UTC
generated hourly time buckets
parsed compound weather data fields

### airports_clean
 - eliminate duplicates
 - select relevant columns
 - evaluate airport codes and time zones for completeness

In [72]:
# check for duplicates
if VALIDATE:
  unique_id = ["IATA", "ICAO", "Tz"]
  w = Window.partitionBy(unique_id)
  display(airports_raw.select('*', f.count("*")\
                      .over(w).alias('dupCount'))\
                      .where('dupCount > 1')\
                      .drop('dupCount'))
# OK => no duplicates 

In [73]:
# DROP DUPLICATES
airports_tmp = airports_raw.dropDuplicates(subset = ["IATA", "ICAO", "Tz"])
airports_tmp.count()

Out[40]: 7698

In [74]:
# FILTER & SELECT RELEVANT COLUMNS
airports_tmp = airports_tmp.select("IATA", "ICAO", "Name", "Tz")
airports_tmp.createOrReplaceTempView("airports_tmp")
airports_tmp.count()

Out[41]: 7698

In [75]:
# check for nulls/missing values
if VALIDATE:
  df = check_nulls_nans(airports_tmp)
  display(df)
  
# there appears to be 1 IATA with null/NaN

In [76]:
# Investigating IATA = null/Nan...
if VALIDATE:
  display(airports_raw.filter(f.isnan('IATA') | f.isnull('IATA')))

# This is not a US aiport. Since we are not going to have flights out of this airport, we can leave as is.
# What's interesting is that 'NAN' is not a missing value NaN - it is infact a legit IATA code! Would be interesting to handle if we are predicting delays for flights out of Fiji :-)

In [77]:
# check for outliers
if VALIDATE:
  display(airports_tmp.describe())
  
# there appear to be some \Ns in IATA, ICAO and Tz fields which are invalid values

In [78]:
# Investigating Tz = '\N'...
if VALIDATE:
  tmp = airports_tmp.select("*").where("length(Tz) = 2")
#   display(tmp)
  print(f"airports with invalid Tzs: {tmp.count()}")
  
# there seem to be 1021 airports with Tz = '\N'

In [79]:
# Investigating the impact of Tz = '\N'...
# Do we have any airport origins that would map to IATAs with Tz = '\N'? 
if VALIDATE:
  airports_with_invalid_Tz = airports_tmp.select("*").where("length(Tz) = 2")
  flight_origins = airlines_raw.select("ORIGIN").distinct()
  origins_with_invalid_Tzs = (flight_origins.withColumnRenamed("ORIGIN", "IATA")
                                 .join(airports_with_invalid_Tz, how="inner", on="IATA")
                           )
  print(f"airports with Tz='\\N': {airports_with_invalid_Tz.count()}")
  print(f"distinct airport origins: {flight_origins.count()}")
  print(f"airport origins with invalid Tz: {origins_with_invalid_Tzs.count()}")
  
  # there seem to be no airports that we have in the 3 year data that have Tz = '\N' . Therefore no impact.

In [80]:
# Investigating the impact of Tz = '\N'...
if VALIDATE:
  display(airports_tmp.select("*").where("length(Tz) = 2 and Country = 'United States'"))
  
# All such seem to be local/municipal airports
# ASSUMPTION: we will not be asked to predict out of such airports

In [81]:
# HANDLE Tz = '\N'
# In the event Tz = '\N' or null, let's default to eastern time
airports_tmp = ( airports_tmp.withColumn("Tz2", f.when(airports_tmp.Tz.isNull() | (airports_tmp.Tz == '\\N'), "America/New_York").otherwise(airports_tmp.Tz))
                      .drop('Tz')
                      .withColumnRenamed("Tz2", "Tz")
                     )
if VALIDATE:
  null_Tz_df = airports_tmp.filter(f.isnan('Tz') | f.isnull('Tz'))
  invalid_Tz_df = airports_tmp.select("*").where("length(Tz) = 2")
  print(f"Null Tzs: {null_Tz_df.count()}")
  print(f"Invalid Tzs: {invalid_Tz_df.count()}")

In [82]:
#Investigating IATAs = '\N'...
if VALIDATE:
  tmp = airports_tmp.select("*").where("length(IATA) != 3")
  print(f"airports with invalid IATA: {tmp.count()}")

In [83]:
#Investigating IATAs = '\N'...
if VALIDATE:
  tmp = airports_tmp.select("*").where("length(IATA) != 3")
  display(tmp)
  
# these seem to all be municipal/private/international airports with no official IATA designation from the FAA.
# there is no way to "create" any designations for these
# investigate further to see if there are any flights from origin airports such as these in the airlines data

In [84]:
# Investigating IATA = '\N' further...
# Checking to see if there are any origin airports that cannot be mapped to IATA's in airports.dat that fall under a similar category as above
if VALIDATE:
  origin_airports = airlines_raw.select("ORIGIN").distinct()
  airports_IATAs = airports_tmp.select("IATA").distinct()
  join_df = origin_airports.withColumnRenamed("ORIGIN", "IATA").join(airports_IATAs, how="left", on="IATA")
  display(join_df.select("IATA").where("length(IATA) != 3"))
  

# there appear to be no flights out of airports with IATA='\N', therefore no need to handle.
# ASSUMPTION: we will not be asked to predict delays for such airports


In [85]:
#Investigating ICAO = '\N'...
if VALIDATE:
  tmp = airports_tmp.select("*").where("length(ICAO) == 2")
  display(tmp)

  # in Brazil. No need to handle.

In [86]:
# DROP IATA='\N' & ICAO= '\N'
airports_tmp = airports_tmp.filter("length(IATA) > 2")
airports_tmp = airports_tmp.filter("length(ICAO) > 2")
airports_tmp = airports_tmp.filter("IATA != 'NAN'")
airports_tmp.count()

Out[53]: 6071

In [87]:
# CREATE AIRPORTS CLEAN
airports_clean = airports_tmp
airports_clean.createOrReplaceTempView("airports_clean")
print(f"airports_clean: {airports_clean.count()} rows")

airports_clean: 6071 rows

In [88]:
# VALIDATE AIRPORTS CLEAN
# check for outliers
if VALIDATE:
  display(airports_clean.describe())

In [89]:
# check for nulls/missing values
if VALIDATE:
  df = check_nulls_nans(airports_clean)
  display(df)

In [90]:
# DISPLAY DATA
if VERBOSE:
  display(airports_clean)

### airlines_clean
 - eliminate duplicates
 - select relevant columns
 - evaluate and handle nulls

In [92]:
# check for duplicates
if VALIDATE:
  unique_id = ["ORIGIN", "DEST", "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM", "FL_DATE"]
  w = Window.partitionBy(unique_id)
  duplicate_flights = airlines_raw.select('*', f.count("*")\
                      .over(w).alias('dupCount'))\
                      .where('dupCount > 1')\
                      .drop('dupCount')
  display(duplicate_flights)
  
# OK => no duplicates  
# if any are listed, drop them

In [93]:
# ELIMINATE DUPLICATES/MULTIPLES
unique_id = ["ORIGIN", "DEST", "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM", "FL_DATE"]
w = Window.partitionBy(unique_id)
airlines_tmp = airlines_raw.select('*', f.count("*")\
                      .over(w).alias('dupCount'))\
                      .where('dupCount = 1')\
                      .drop('dupCount')
airlines_tmp.createOrReplaceTempView("airlines_tmp")
airlines_tmp.count()


Out[59]: 31746811

In [94]:
# SELECT RELEVANT COLUMNS
airline_relevant_columns = ['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 
                            'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 
                            'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR',
                            'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
                            'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 
                            'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 
                            'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'AIR_TIME',
                            'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 
                            'CRS_ELAPSED_TIME', 
                            'FLIGHTS', 'holiday',
                            'DISTANCE', 'DISTANCE_GROUP', 
                            'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']


airlines_tmp = airlines_tmp.select(airline_relevant_columns) 
airlines_tmp.count()

Out[60]: 31746811

In [95]:
# check for outliers
if VALIDATE:
  display(airlines_tmp.describe())
  
# there appear to be no outliers in the "primary" fields we care about
# TAXI_IN, TAXI_OUT, WHEEL_ON, WHEELS_OFF have figures a few 100s of minutes - seems fishy - careful if/when using as feature.
# different counts for different fields => missing or null values

In [96]:
# VALIDATE AIRLINES DATA
# check for nulls/missing values
if VALIDATE:
  df = check_nulls_nans(airlines_tmp)
  display(df)
  
# lot of fields with nulls and nans

In [97]:
# CREATE AIRLINES CLEAN
airlines_clean = airlines_tmp
airlines_clean.createOrReplaceTempView("airlines_clean")
if VALIDATE:
  print(f"airlines_clean: {airlines_clean.count()} rows") 

In [98]:
# VALIDATE AIRLINES CLEAN
# check for outliers
if VALIDATE:
  display(airlines_clean.describe())


In [99]:
# check for nulls/missing values
if VALIDATE:
  df = check_nulls_nans(airlines_clean)
  display(df)
  
# Should be all zeroes as all nulls have been accounted for

In [100]:
# print schema
if VERBOSE:
  airlines_clean.printSchema()

In [101]:
# DISPLAY DATA
if VERBOSE:
  display(airlines_clean)

[Slide10-Sirak]

####Joins

[NS7]As a part of joins, we
 - First joined airlines data with airports. The ORIGIN and DEST fields in airlines data mapped to the IATA field in the airports table
 - Next we joined weather data to airports data. The CALL_SIGN in weather data mapped to the ICAO of the airports data
 - Our big join consisted of joining the results of the above two joins.
 
The results of these joins were saved as our Silver Data.
 
 - We then joined on the aggregate delay data. This data was much smaller, since it has at most one record for each (airport, hour)

#### Join1

### airlines_silver (airlines join airports)
 - join airlines data with airports data, such that every airport has an associated ICAO code

In [106]:
# before the join, ensure that every ORIGIN in airlines_clean has a corresponding IATA, ICAO and Tz in airports_clean
# missing values imply that:
# 1 - there is no entry in airports.dat for ORIGIN airport, or
# 2 - the ICAO is coded incorrectly for the airport (should not be the case if airports_clean is really clean)
if VALIDATE:
  origins = airlines_clean.select("ORIGIN").distinct()
  destinations = airlines_clean.select("DEST").distinct()
  airports_iata = airports_clean.select("IATA").distinct()
  
  print(f"flight origins: {origins.count()}")
  print(f"flight destinations: {destinations.count()}")
  print(f"airports with IATA: {airports_iata.count()}")
  print(f"airports: {airports_clean.count()}")

In [107]:
# Investigating further...
# ORIGINs in airlines_clean with no matching IATA in airports_clean
if VALIDATE:
  missing = (airlines_clean.select("ORIGIN").distinct()).subtract(airports_clean.select("IATA").distinct())
  print(f"Origin airports with no matching IATAs: {missing.count()}")

In [108]:
# Investigating further...
# list of missing IATAs
if VALIDATE:
  flights = airlines_clean.join(missing, on="ORIGIN", how="inner")
  display(flights.select("ORIGIN", "ORIGIN_CITY_NAME").distinct())

In [109]:
# Investigating further...
# flights with origins that have no matching IATA
if VALIDATE:
  flights = airlines_clean.join(missing, on="ORIGIN", how="inner")
  print(f"Flights with no matching IATA in airports_clean: {flights.count()}")


[Slide6-Max]

####Time Normalization 

Weather data has timestamps which are unambiguous representations of instants, independent of time zone, but times in the airlines dataset are awkwardly represented as decimal numbers and textual days: departure times are in the time zone of the origin airport and arrivals are in the time zone of the destination. We chose to normalize all these times to timestamps by converting them to UTC. We did this by using spark builtin conversion, but this concealed some details, such as daylight vs. standard time.

We used a publicly available airports database to get the time zone of airports and to assist in matching airline codes (IATA vs ICAO). This dataset covers almost all the airports that are referenced in the "airlines" (flights) dataset, but we needed to impute values for some missing airports.

In [112]:
# FUNCTION TO JOIN AIRLINES AND AIRPORTS DATA
# ensure LEFT JOIN between airlines and airport codes
# we want to make sure every airport is associated with an IATA from airports

def make_airlines_join_airports(airlinesDF, airportsDF):
  airport_codes_origin = airportsDF
  airport_codes_dest = airportsDF

  # update ICAO and universal timezone (Tz) and timezone offset (Timezone) for all origin aiports
  df1 = ( airlinesDF.withColumnRenamed("ORIGIN", "IATA")
        .join(airport_codes_origin, on="IATA", how="left")
        .withColumnRenamed("IATA", "ORIGIN")
        .withColumnRenamed("ICAO", "origin_icao")
        .withColumnRenamed("Tz", "origin_time_zone")
        .withColumnRenamed("Timezone", "origin_time_zone_offset")
        .withColumnRenamed("Name", "airport_name")
     )
  
  # update ICAO and universal timezone (Tz) and timezone offset (Timezone) for all destination aiports
  df2 = ( df1.withColumnRenamed("DEST", "IATA")
        .join(airport_codes_dest, on="IATA", how="left")
        .withColumnRenamed("IATA", "DEST")
        .withColumnRenamed("ICAO", "dest_icao")
        .withColumnRenamed("Tz", "dest_time_zone")
        .withColumnRenamed("Timezone", "dest_time_zone_offset")
     )
  
  # convert arrival and departure times to UTC format
  # only crs_dep_time is needed
  df3 = ( df2
         .withColumn("crs_dep_time_utc", f.to_utc_timestamp(f.format_string("%s %02d:%02d", 
                                                    df2.FL_DATE, 
                                                    (df2.CRS_DEP_TIME / 100).cast(IntegerType()), 
                                                    df2.CRS_DEP_TIME % 100), 
                                                     df2.origin_time_zone))
        .withColumn("crs_arr_time_utc", f.to_utc_timestamp(f.format_string("%s %02d:%02d", 
                                                   df2.FL_DATE, 
                                                   (df2.CRS_ARR_TIME / 100).cast(IntegerType()), 
                                                   df2.CRS_ARR_TIME % 100), 
                                                    df2.origin_time_zone))
        .withColumn("dep_time_utc", f.to_utc_timestamp(f.format_string("%s %02d:%02d", 
                                                   df2.FL_DATE, 
                                                   (df2.DEP_TIME / 100).cast(IntegerType()), 
                                                   df2.DEP_TIME % 100), 
                                                    df2.origin_time_zone))
        .withColumn("arr_time_utc", f.to_utc_timestamp(f.format_string("%s %02d:%02d", 
                                                   df2.FL_DATE, 
                                                   (df2.ARR_TIME / 100).cast(IntegerType()), 
                                                   df2.ARR_TIME % 100), 
                                                   df2.origin_time_zone))
        )
  
  # add time buckets to all other time related fields
  # Max: time buckets are only used for aggregation and join - they are not features. so we don't need to add buckets for all times
  df4 = ( df3
        .withColumn("CRS_DEP_HOUR", (df3.CRS_DEP_TIME / 100).cast(IntegerType()))  # possible feature
        .withColumn("CRS_ARR_HOUR", (df3.CRS_ARR_TIME / 100).cast(IntegerType()))  # possible feature 
        # MAX: sch_dep_time_bucket could be used to join aggregates about scheduled traffic
        .withColumn("sch_dep_time_bucket", f.date_format(f.date_trunc("Hour", df3.crs_dep_time_utc), "yyyy-MM-dd HH:mm"))
        .withColumn("sch_arr_time_bucket", f.date_format(f.date_trunc("Hour", df3.crs_arr_time_utc), "yyyy-MM-dd HH:mm"))
        .withColumn("act_dep_time_bucket", f.date_format(f.date_trunc("Hour", df3.dep_time_utc), "yyyy-MM-dd HH:mm"))
        .withColumn("act_arr_time_bucket", f.date_format(f.date_trunc("Hour", df3.arr_time_utc), "yyyy-MM-dd HH:mm"))
        # MAX: pred_time_bucket is used to join weather reports and aggregated previous delays
        .withColumn("pred_time_bucket", f.date_format(f.date_trunc("Hour", df3.crs_dep_time_utc - f.expr("INTERVAL 2 HOUR")), "yyyy-MM-dd HH:mm"))
       )
  
  return df4
      

In [113]:
# CREATE AIRLINES_SILVER
airlines_silver = make_airlines_join_airports(airlines_clean, airports_clean)
airlines_silver.createOrReplaceTempView("airlines_silver")
if VALIDATE:
  print(f"airlines_silver: {airlines_silver.count()} rows")

In [114]:
# VALIDATE AIRLINES_SILVER
# check for duplicates
if VALIDATE:
  unique_id = ["ORIGIN", "DEST", "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM", "FL_DATE"]
  w = Window.partitionBy(unique_id)
  display(airlines_silver.select('*', f.count("*")\
                         .over(w).alias('dupCount'))\
                         .where('dupCount > 1')\
                         .drop('dupCount'))
# OK => no duplicates  

In [115]:
# Evaluate whether every airport code has a valid ICAO
if VALIDATE:
  airlines_silver.select("*").where("origin_icao is null").count()
  airlines_silver.select("*").where("origin_time_zone is null").count()

# counts must be zero

In [116]:
# check for nulls/nans
if VALIDATE:
  display(check_nulls_nans(airlines_silver))

In [117]:
# cross-check number of records with airlines_clean
if VALIDATE:
  silver_count = airlines_silver.count()  
  clean_count = airlines_clean.count()
  print(f"silver_count {silver_count}")
  print(f"clean_count {clean_count}")  
  assert(silver_count == clean_count)

In [118]:
# check outliers
# Ensure no missing values - everything should be accounted for
if VERBOSE:
  display(airlines_silver.describe())

In [119]:
# print schema
if VERBOSE:
  airlines_silver.printSchema()

In [120]:
# display data
if VERBOSE:
  display(airlines_silver)

### weather_clean
- filter for US
- split fields where required
- convert dates to UTC (are they already in?) 
- add time bucket
- handle nulls
- compact multiple rows for the same time bucket
- interpolate missing measurements

In [122]:
# FILTER WEATHER DATA
# US only 
weather_us = weather_raw.filter((f.col('report_type') == 'FM-15') & (f.col('call_sign') != '99999'))
weather_us.createOrReplaceTempView("weather_us")
if VALIDATE:
  print(f"weather_us: {weather_us.count()} rows")

In [123]:
# verify counts of valid US stations (must equal ~2166)
if VALIDATE:
  display(weather_us.select(f.countDistinct("call_sign")))

[Slide7-Max]

####Time Buckets

Since there are many events in the system that we may want to join and aggregate, and these may all occur at slightly different timestamps, we introduced an idea of time "buckets". These are time intervals into which different events can be aggregated and by which aggregations can be joined. We chose hour-wide buckets, similar to the "TIME_BLK" fields of the airlines data, but unified to UTC time. These time buckets were to be used only as handles for data engineering and aggregation; they are not "features" in the ML sense - whereas the local TIME_BLK may be a feature that has predictive value.

We believed it is crucial to consider only data that was available at the prediction time, which is two hours before the flight departure time. We added a prediction time bucket to each flight, which we used as the join key to the weather data.

In [126]:
# TRANSFORMATIONS
# pre-split wind and temperature fields and handle missing values
# apply scale refactoring as noted in ISD documentation
# add time bucket 
# rename columns
# collapse multiple measurements within a time bucket to a single one

def make_weather_clean(weatherDF):

  wind_split = f.split(weatherDF.WND, ',')
  tmp_split = f.split(weatherDF.TMP, ',')
  dew_split = f.split(weatherDF.DEW, ',')
    
    
  # Add one-hot encoded flags to indicate missing entries for wind, windangle, tmp and dew
  # Wind angle is the exception, only considered missing if windspeed was also missing
  weather_tmp = ( weatherDF  
                .withColumn("CALL_SIGN", trim(weatherDF.CALL_SIGN))
                .withColumn("miss_windAngle", f.when(wind_split.getItem(3) == "999", 1).otherwise(0))
                .withColumn("windangle", f.when(wind_split.getItem(3) == "0000", 0.)
                                          .otherwise(wind_split.getItem(0).cast("float")) )
                .withColumn("miss_wnd", f.when(wind_split.getItem(3) == "999", 1).otherwise(0))
                .withColumn("WND", wind_split.getItem(3).cast("float") / 10.)
                .withColumn("miss_tmp", f.when(tmp_split.getItem(0) == "+9999", 1).otherwise(0))
                .withColumn("TMP", tmp_split.getItem(0).cast("float") / 10.)
                .withColumn("miss_dew", f.when(dew_split.getItem(0) == "+9999", 1).otherwise(0))
                .withColumn("DEW", dew_split.getItem(0).cast("float") / 10.)
                .withColumn("CIG", f.split(weatherDF.CIG, ',').getItem(0).cast("float"))
                .withColumn("VIS", f.split(weatherDF.VIS, ',').getItem(0).cast("float"))
                .withColumn("time_bucket", f.date_format(f.date_trunc("hour", "date"), "yyyy-MM-dd HH:mm"))
               )
  
  # collapse multiple weather rows for a given timebucket into a single one
  unique_id = ["CALL_SIGN", "time_bucket"]
  w = Window.partitionBy(unique_id).orderBy(f.desc('DATE'))
  weather_tmp = weather_tmp.withColumn('Rank',f.dense_rank().over(w))
  weather_tmp = weather_tmp.filter(weather_tmp.Rank == 1).drop(weather_tmp.Rank)
  weather_tmp = weather_tmp.select("CALL_SIGN", "DATE", "windangle", "WND", "TMP", "DEW",
                                   "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew",
                                   "CIG", "VIS", "time_bucket", "LATITUDE", "LONGITUDE", "ELEVATION" )
  return weather_tmp




In [127]:
weather_tmp = make_weather_clean(weather_us)
weather_tmp.createOrReplaceTempView("weather_tmp")
if VALIDATE:
  print(f"weather_clean: {weather_tmp.count()} rows")

In [128]:
# VALIDATE TRANSFORMATIONS
# check for existence of multiple measurements with same time bucket - there should be none
if VALIDATE:
  unique_id = ["CALL_SIGN", "time_bucket"]
  w = Window.partitionBy(unique_id)
  display(weather_tmp.select('*', f.count("*")\
                      .over(w).alias('dupCount'))\
                      .where('dupCount > 1')\
                      .drop('dupCount'))
  
  # OK => no duplicates or multiples => no need to collapse

In [129]:
# check if there are any weather stations missing weather data that we care about i.e. CALL_SIGN, time_bucket, WND, windangle, CIG, VIS, TMP, DEW
if VALIDATE:
  df_tmp = weather_tmp.select('CALL_SIGN', 'time_bucket', 'WND', 'windangle', 'CIG', 'VIS', 'TMP', 'DEW',
                              "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew", "LATITUDE", "LONGITUDE", "ELEVATION")
  df = check_nulls_nans(df_tmp)
  display(df)
  
# All zeroes mean no nulls/nans => no need to correct for missing values

In [130]:
# CREATE WEATHER_CLEAN
weather_clean = weather_tmp
weather_clean.createOrReplaceTempView("weather_clean")
if VALIDATE:
  print(f"weather_clean: {weather_clean.count()} rows")

In [131]:
# VALIDATE WEATHER CLEAN
# check for outliers
if VERBOSE:
  display(weather_clean.describe())

In [132]:
# check for nulls/nans
if VALIDATE:
  df_tmp = weather_clean.select('CALL_SIGN', 'time_bucket', 'WND', 'windangle', 'CIG', 'VIS', 'TMP', 'DEW', 
                                "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew", "LATITUDE", "LONGITUDE", "ELEVATION")
  df = check_nulls_nans(df_tmp)
  display(df)

In [133]:
# DISPLAY DATA
if VERBOSE:
  display(weather_clean)

[Slide11-Sirak]

####Join2
[NS8]

### weather_silver (airports join weather)
- join weather data with airport data, such that every ICAO in the airports data is associated with a CALL_SIGN in weather data

In [137]:
# before joining, ensure no nulls in airports_clean
# check for nulls/nans
if VALIDATE:
  display(check_nulls_nans(airports_clean))

In [138]:
# check to make sure there are no airports with ICAO = '\N'
if VALIDATE:
  display(airports_clean.select("*").where("length(ICAO) = 2"))


In [139]:
# before joining, ensure no nulls in weather_clean
# check for nulls/nans
if VALIDATE:
  display(check_nulls_nans(weather_clean))

In [140]:
# FUNCTION TO JOIN AIRPORTS WITH WEATHER
# ensure RIGHT JOIN for weather.join(airports)

def make_weather_silver(weatherDF, airportsDF):
    """ Join weather data with airports
        Weather Data (CALL_SIGN)
        Airports Data (ICAO)
    """
       
    # Join weather_raw to airport_codes based on ICAO
    ## rename CALL_SIGN to ICAO to facilitate join
    ## add a time_bucket field to capture the hour of recording
    ## select relevant columns from weather_raw to minimize working data
    ## inner join eliminates orphan weather records
    df = ( weatherDF.withColumnRenamed("CALL_SIGN", "ICAO")
                  .withColumnRenamed("DATE", "date")
                  .withColumnRenamed("NAME", "station_name")
                  .withColumnRenamed("WND", "wind")
                  .withColumnRenamed("CIG", "cig")
                  .withColumnRenamed("VIS", "vis")
                  .withColumnRenamed("TMP", "tmp")
                  .withColumnRenamed("DEW", "dew")
                  .withColumnRenamed("LATITUDE", "LATITUDE")
                  .withColumnRenamed("LONGITUDE", "LONGITUDE")
                  .withColumnRenamed("ELEVATION", "ELEVATION")
                  .select("time_bucket", "ICAO", "wind",  "windangle", "cig", "vis", "tmp", "dew",
                          "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew", "LATITUDE", "LONGITUDE", "ELEVATION")
                  .join(airportsDF, on="ICAO", how="right")
          )
    return df

In [141]:
# CREATE WEATHER_SILVER
weather_silver = make_weather_silver(weather_clean, airports_clean)
weather_silver.createOrReplaceTempView("weather_silver")
if VALIDATE:
  print(f"weather_silver: {weather_silver.count()} rows")


In [142]:
# check for duplicates
if VALIDATE:
  unique_id = ["ICAO", "time_bucket"]
  w = Window.partitionBy(unique_id)
  display(weather_silver.select('*', f.count("*")\
                        .over(w).alias('dupCount'))\
                        .where('dupCount > 1')\
                        .drop('dupCount'))
# OK => no duplicates 


In [143]:
# check for nulls/missing values - there should be none - should have all been handled by this point
if VALIDATE:
  df = check_nulls_nans(weather_silver)
  display(df)  

# all weather related fields must be zeroes
# if weather related fields are non-zero, these imply airports with no weather stations

In [144]:
# DESCRIPTIVE STATISTICS
if VALIDATE:
  # 2.63 minutes for 2015_2017
  display(weather_silver.describe())

In [145]:
# PRINT SCHEMA
if VERBOSE:
  weather_silver.printSchema()

In [146]:
# DISPLAY DATA
if VERBOSE:
  display(weather_silver)

## Silver (joined) Dataset

airports_silver joined with weather_silver

[Slide12-Sirak]

####Join3

In [150]:
# FUNCTION TO JOIN AIRLINES AND WEATHER 
def make_airlines_join_weather(airlinesDF, weatherDF):
  # join airlines_silver and weather_silver on origin airport (origin_icao) and prediction time bucket (time_bucket)
  join_columns = ["ICAO","time_bucket"]
   
  # join airlines and weather data on origin airport 
  df = ( airlinesDF
        .withColumnRenamed("origin_icao", "ICAO")
        .withColumnRenamed("pred_time_bucket", "time_bucket")
        .join(weatherDF, join_columns, how="left")
        .withColumnRenamed("time_bucket", "pred_time_bucket")
        .withColumnRenamed("ICAO", "origin_icao")
       )
  return df



In [151]:
# (Madhukar) Identify common columns that need to be dropped after joining
columns_to_drop = list(set(airlines_silver.columns).intersection(set(weather_silver.columns)))
print("Drop these duplicated columns", columns_to_drop)

# CREATE SILVER DATA
silver_df = make_airlines_join_weather(airlines_silver, weather_silver).drop(weather_silver.Name)
silver_df.createOrReplaceTempView("silver_df")
if VALIDATE:
  airlines_silver_count = airlines_silver.count()  
  silver_df_count = silver_df.count()
  print(f"airlines_silver_count {airlines_silver_count}")
  print(f"silver_df_count {silver_df_count}")  
  assert(airlines_silver_count == silver_df_count)

Drop these duplicated columns ['Name']

In [152]:
if VALIDATE:
  query = f"""
select count(*) as count, "silver_df" as table from silver_df
union
select count(*), "airlines_silver" from airlines_silver
union
select count(*), "weather_silver" from weather_silver  
"""
  if VERBOSE:
    print(query)
  df = spark.sql(query)
  display(df)

In [153]:
# DESCRIPTIVE STATISTICS - all columns must have equal number of values
if VERBOSE:
  display(silver_df.describe())

In [154]:
# check for nulls/missing values - there should be none - should have all been handled by this point
if VALIDATE:
  df = check_nulls_nans(silver_df)
  display(df)  


In [155]:
# PRINT SCHEMA
if VERBOSE:
  silver_df.printSchema()

In [156]:
# DISPLAY DATA
if VERBOSE:
  display(silver_df)

## Gold Datasets
These datasets refelect datasets ready for training our models
* Gold_Train_Validate (2015-2017)
* Gold_Test (2018)
* once everything has been tested and is successful, we bring in 2019 via the same pipeline

[Slide8-Max]

####Airport Busyness Aggregation

We speculated that previous delays at an airport could contribute to further delays, so we developed aggregations that express delays, as evidenced by actual flights delayed, and planned busyness, as expressed by scheduled activity. Although these two kinds of measures are both aggregated by bucket for each airport, we were very cautious about how we join these aggregates. Scheduled activity in a given bucket can be joined with flights that depart in that bucket, since schedules are known in advance, but aggregate actual delay must be associated with the prediction time bucket, since at prediction time, we can only predict on the basis of the delays that have happened before prediction time. Moreover, since the full actual delays in minutes are not known until the flight actually takes off, we must aggregate actual delays by actual departure time. This might cause bad predictiveness in the case of very long delayed flights.

##1. Departure Delay aggregates

In [161]:
def compute_airport_aggregated_delay(verbose=True):
  """
  Create a time able that summarises average delays of flights that
  departed in a given hour
  
  input: silver_df
  
  output: df with hourly delay aggregates:
  prior_count_DEP_TIME, 
  prior_avg_DEP_DELAY, 
  prior_agg_DEP_DEL15, 
  prior_avg_TAXI_OUT, 
  prior_avg_CARRIER_DELAY, 
  prior_avg_WEATHER_DELAY, 
  prior_avg_NAS_DELAY,
  prior_avg_SECURITY_DELAY, 
  prior_avg_LATE_AIRCRAFT_DELAY, 
  prior_DEP_OVERFLOW (this cant be calculated here b)
  as difference between planned and actual departures
  """
  
  table_name = 'silver_df'
  query = f"""
select ORIGIN, 
           act_dep_time_bucket as time_bucket,
           count(DEP_TIME) as prior_count_DEP_TIME, 
           avg(DEP_DELAY) as prior_avg_DEP_DELAY,            
           sum(DEP_DEL15) as prior_agg_DEP_DEL15, 
           avg(TAXI_OUT) as prior_avg_TAXI_OUT, 
           avg(CARRIER_DELAY) as prior_avg_CARRIER_DELAY, 
           avg(WEATHER_DELAY) as prior_avg_WEATHER_DELAY, 
           avg(NAS_DELAY) as prior_avg_NAS_DELAY, 
           avg(SECURITY_DELAY) as prior_avg_SECURITY_DELAY, 
           avg(LATE_AIRCRAFT_DELAY) as prior_avg_LATE_AIRCRAFT_DELAY
         from {table_name}
       group by ORIGIN, 
                act_dep_time_bucket 
    """

  if verbose:
    print(query)
    df = spark.sql(query)
    return df

  
airport_aggregated_delay = compute_airport_aggregated_delay().cache()
airport_aggregated_delay.createOrReplaceTempView("airport_aggregated_delay")

select ORIGIN, 
 act_dep_time_bucket as time_bucket,
 count(DEP_TIME) as prior_count_DEP_TIME, 
 avg(DEP_DELAY) as prior_avg_DEP_DELAY, 
 sum(DEP_DEL15) as prior_agg_DEP_DEL15, 
 avg(TAXI_OUT) as prior_avg_TAXI_OUT, 
 avg(CARRIER_DELAY) as prior_avg_CARRIER_DELAY, 
 avg(WEATHER_DELAY) as prior_avg_WEATHER_DELAY, 
 avg(NAS_DELAY) as prior_avg_NAS_DELAY, 
 avg(SECURITY_DELAY) as prior_avg_SECURITY_DELAY, 
 avg(LATE_AIRCRAFT_DELAY) as prior_avg_LATE_AIRCRAFT_DELAY
 from silver_df
 group by ORIGIN, 
 act_dep_time_bucket

In [162]:
def compute_airport_aggregates(verbose=True):
  """
  Create a time able that summarises aggregates of flights that
  departed in a given hour
  
  input: silver_df
  
  output: df with hourly aggregates:
  count_CRS_DEP_TIME
  """
  
  table_name = 'silver_df'
  query = f"""
select ORIGIN, sch_dep_time_bucket,
           count(CRS_DEP_TIME) as count_CRS_DEP_TIME
         from {table_name}
       group by ORIGIN, 
                sch_dep_time_bucket 
    """

  if verbose:
    print(query)
    df = spark.sql(query)
    return df

  
airport_aggregates = compute_airport_aggregates().cache()
airport_aggregates.createOrReplaceTempView("airport_aggregates")

select ORIGIN, sch_dep_time_bucket,
 count(CRS_DEP_TIME) as count_CRS_DEP_TIME
 from silver_df
 group by ORIGIN, 
 sch_dep_time_bucket

### Join departure aggregated delays with silver_df

[Slide13-Sirak]

####Join4

In [166]:
# join departure aggregates
def join_airport_agg_and_silver(airport_aggregated_delay, silver_df):
  # rename columns to facilitate join
  agg_df = (airport_aggregated_delay
              .withColumnRenamed("time_bucket", "pred_time_bucket")
              .select("pred_time_bucket", "ORIGIN", "prior_count_DEP_TIME",
                      "prior_avg_DEP_DELAY", "prior_agg_DEP_DEL15", "prior_avg_TAXI_OUT",
                      "prior_avg_CARRIER_DELAY", "prior_avg_WEATHER_DELAY", "prior_avg_NAS_DELAY",
                      "prior_avg_SECURITY_DELAY", "prior_avg_LATE_AIRCRAFT_DELAY")
            )
              
  res = (silver_df
                  .withColumnRenamed("time_bucket", "pred_time_bucket")
                  .join(agg_df, on=['ORIGIN', 'pred_time_bucket'], how="left") )
  return res

gold_df_tmp_joined = join_airport_agg_and_silver(airport_aggregated_delay, silver_df)

In [167]:
# add scheduled hourly aggregate count_CRS_DEP_TIME and join it
def add_hourly_agg(airport_aggregates, gold_df_tmp_joined):
  # rename columns to facilitate join
  hourly_agg_df = (airport_aggregates
              .select("sch_dep_time_bucket", "ORIGIN", "count_CRS_DEP_TIME")
            )
              
  res = (gold_df_tmp_joined
                  .join(hourly_agg_df, on=['ORIGIN', 'sch_dep_time_bucket'], how="left") )
  return res

gold_df_tmp_joined = add_hourly_agg(airport_aggregates, gold_df_tmp_joined)

In [168]:
if VALIDATE:
  gold_df_tmp_count = gold_df_tmp_joined.count()  
  clean_count = airlines_clean.count()
  print(f"gold_df_tmp_count {gold_df_tmp_count}")
  print(f"clean_count {clean_count}")  
  assert(gold_df_tmp_count == clean_count)

## 2. Tracking flight delays in previous leg using TAIL_NUM

In [170]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead

window_w_offset = (Window.partitionBy('TAIL_NUM', 'FL_DATE').orderBy('dep_time_utc'))

# if null, impute 2 (so that prior_TN_DEP_DEL15 is OHE by 0, 1 and 2)
gold_df_tmp_joined = (gold_df_tmp_joined.withColumn('prior_TN_DEP_DEL15', lag(col('DEP_DEL15'), 1, 2).over(window_w_offset))).cache()


# introduce AIR_TIME_FLAG that flags 1 for >120min (that is, this flag is 0 for prior_TN_DEP_DEL15=[0,1], and 1 for prior_TN_DEP_DEL15=2)
gold_df_tmp_joined = ( gold_df_tmp_joined
                .withColumn("AIR_TIME_FLAG", f.when(gold_df_tmp_joined.AIR_TIME>120, 1).otherwise(0))
               )


gold_df_tmp_joined.createOrReplaceTempView("gold_df_tmp_joined")

In [171]:
# replace prior_TN_DEP_DEL15 by 2 for flights <120min duration (so that prior_TN_DEP_DEL15 is OHE by 0, 1 and 2)
from pyspark.sql import functions as F
gold_df_tmp_joined = gold_df_tmp_joined.withColumn("prior_TN_DEP_DEL15", F.when(F.col("AIR_TIME")<120, 2).otherwise(F.col("prior_TN_DEP_DEL15")))
gold_df_tmp_joined.createOrReplaceTempView("gold_df_tmp_joined")

[Slide14-Rad]

### Feature Engineering
Input feature attributes for our model were based mostly on intuition and work previously undertaken by researchers. We classified features into the following broad categories:

**Spatial** characteristics: origin, destination

**Temporal** characteristics: month, day of week, time of day

**Flight Performance** characteristics: planned arrival, planned departure schedules 

**Weather** characteristics: represent external and environmental conditions

**State of the System** characteristics: cumulative delays by category, number of flights departing in the hour, runway performance (taxi in/out) and late arrivals

In [174]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/feature%20selection.png?token=AKFL6AAT5A5LJQDRA4CWF427GVXAY' >")

Most were readily available in the Silver data while the remaining new features were aggregated separately and joined with the Silver data. As previously pointed out, time buckets were used to both generate these counts as well as append them as features of each of the flights.

We also noted that correlated and irrelevant features may provide model overfitting and decrease prediction performance, and analysing feature importance may enable us to discriminate and study the most impactful features. Although we did not have a chance to implement this in the current approach, we plan to do so in our next iteration. 

Next, all categorical features were identified and converted to strings and all numeric variables were converted to doubles and the flights along with selected features were saved as our GOLD data.

#### *Convert Categorical Variables to Strings, and Numericals to Doubles

In [177]:
# identify all categorical columns and convert them to string, and convert all numerical columns to doubles

full_categorical_features = ['ORIGIN', 'sch_dep_time_bucket', 'ICAO', 'pred_time_bucket', 'DEST', 'YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM', 'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC', 'CRS_DEP_TIME', 'DEP_TIME',  'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'WHEELS_OFF', 'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',  'DISTANCE_GROUP', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'airport_name',  'origin_time_zone', 'dest_icao', 'dest_time_zone', 'crs_dep_time_utc', 'crs_arr_time_utc', 'dep_time_utc', 'arr_time_utc', 'sch_arr_time_bucket', 'act_dep_time_bucket', 'act_arr_time_bucket', 'station_name', 'IATA', 'Name', 'Timezone', 'Tz', 'ORIGIN-DEST', 'prior_TN_DEP_DEL15', 'holiday', 'AIR_TIME_FLAG'
                            ]

full_numerical_features = ['DEP_DELAY', 'DEP_DELAY_NEW', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'ARR_DELAY_NEW', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'origin_time_zone_offset', 'dest_time_zone_offset', 'CRS_DEP_HOUR', 'CRS_ARR_HOUR', 'wind', 'windangle', 'cig', 'vis', 'tmp', 'dew', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'count_CRS_DEP_TIME', 'prior_count_DEP_TIME', 'prior_agg_DEP_DEL15', 'prior_avg_DEP_DELAY', 'prior_avg_TAXI_OUT', 'prior_avg_CARRIER_DELAY', 'prior_avg_WEATHER_DELAY', 'prior_avg_NAS_DELAY', 'prior_avg_SECURITY_DELAY', 'prior_avg_LATE_AIRCRAFT_DELAY', "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew", 'prior_dep_agg_w_overflow'
                          ]

from pyspark.sql.functions import expr, col, column

def cast_features(df, full_categorical_features, full_numerical_features):
  for cat_feature in full_categorical_features:
    if cat_feature in df.columns:
      df = (df.withColumn(cat_feature, col(cat_feature).cast("string")))
    
  for num_feature in full_numerical_features:
    if num_feature in df.columns:
      df = (df.withColumn(num_feature, col(num_feature).cast("double")))
  
  return df

gold_df_tmp_cast = cast_features(gold_df_tmp_joined, full_categorical_features, full_numerical_features)

In [178]:
if VERBOSE :
  gold_df_tmp_cast.count()
  gold_df_tmp_cast.printSchema()

In [179]:
# identify the features that need to be saved in gold_df
feature_list = ['ORIGIN', 'DEST', 'ORIGIN-DEST', 
                'YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'CRS_DEP_HOUR', 
                'FL_DATE', 'TAIL_NUM', 'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP',
                'wind', 'windangle', 'cig', 'vis', 'tmp', 'dew', "miss_windAngle", "miss_wnd", "miss_tmp", "miss_dew", 
                'LATITUDE', 'LONGITUDE', 'ELEVATION', 'holiday',
                'DEP_DEL15', 
                'count_CRS_DEP_TIME', 'prior_count_DEP_TIME', 'prior_agg_DEP_DEL15', 
                'prior_avg_DEP_DELAY', 'prior_avg_TAXI_OUT', 
                'prior_avg_CARRIER_DELAY', 'prior_avg_WEATHER_DELAY', 'prior_avg_NAS_DELAY', 
                'prior_avg_SECURITY_DELAY', 'prior_avg_LATE_AIRCRAFT_DELAY', 'prior_TN_DEP_DEL15', 'AIR_TIME_FLAG'
               ]




In [180]:
def df_trim(old_df, feature_list, keep_features=True):
  """
  returns trimmed DF that only keeps (if True, else drops) columns in features_list
  
  Input: original dataframe, list of features, keep features=True will keep only these features
  Output: trimmed dataframe
  """

  if keep_features:
    del_feature_list = set(list(silver_df.columns)) - set(feature_list)
  else:
    del_feature_list = feature_list
    
  trimmed_df = old_df.drop(*del_feature_list) # drop all columns in del_feature_list
  return trimmed_df

gold_df = df_trim(gold_df_tmp_cast, feature_list, True).cache() # this is needed to get rid of columns (like DIV1 etc) that have all nulls
gold_df.createOrReplaceTempView("gold_df")

In [181]:
if VERBOSE :
  gold_df.printSchema()

#### *Save Gold Data

In [183]:
# SAVE GOLD DATASET TO DISK
gold_data_path = f"/team7/{user_initials}/{DATASETS}/gold_dataset"

def persist_gold(df):
  # SAVE GOLD DATASET TO DISK
  (df.write.mode("overwrite").format("parquet").save(gold_data_path))

def init_gold_from_disk():
  # IMPORT GOLD DATA FROM DISK
  print("importing gold data from disk")
  df = sqlContext.read.option("header", "true").parquet(f"dbfs:" + gold_data_path)
  return df

if not FORCE_COMPUTE_ALL and file_exists(gold_data_path):
  gold_df = init_gold_from_disk()
  print("Gold data already exists at {}!".format(gold_data_path))

gold_df.createOrReplaceTempView("gold_df")

# Model Building

[Slide15-Madhukar]

###Algorithm Exploration
[NS12]We tried Logistic Regression (LR), Random Forest (RF) and Gradient Boosted Trees (GBT), and finally chose LR and GBT. 

Logistic regression returns probabilities of belonging to class 0 or class 1, hence one could set an appropriate threshold that would maximize precision (as opposed to recall). For instance, one could set the decision threshold at a probability of 0.9 for the label to be deemed positive (1, or delay in our case). While we expect logistic regression to be decently performant, it could suffer greatly from overfitting, and hence result in a high variance. 

By choosing ensemble methods like Random Forests and Gradient Boosted Trees, we can trade bias for variance (that is, we want a model that is reasonably robust to incoming/updated training data).
While we really wanted to implement LightGBM and XGBoost, these were not readily available to us - the former could not be loaded onto the cluster due to issues with the third party Microsoft Azure library, while the latter was available only in Scala. As a compromise, we chose GBT. 

In order to further improve model performance, we used stacking by taking the predictions of logistic regression and GBT and passing them through another meta-estimator GBT. We found that the performance of the stacked model was the best overall, followed by GBT and then LR.

- Tried Logistic Regression (LR), Random Forest (RF) and Gradient Boosted Trees (GBT)
- **Trade bias for variance using ensemble methods** like GBTs
- LightGBM and XGBoost were not readily available to us
- In order to further improve model performance, we used **stacking of LR and GBT with GBT as meta-estimator**

d ###Algorithm Implementation

In [189]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/Model%20Pipeline.png?token=AFP5XG3ZQLVXWJHGK3A5HTS7GWM4U' width=900 height=350>")

GBT is an effective ensemble learning algorithm based on the idea of boosting. It’s easier to explain for the case of regression, than for classification. A first decision tree regressor is built. Then, its residuals are used to train a second tree (after multiplying the residuals with a learning rate). Then a third tree is built based on the new residuals, and so on. Boosting comes from the learning rate, which is a hyperparameter, and helps to reduce bias. Thus GBT can greatly overfit leading to high variance, and therefore the depth of trees as well as the total number of boosted trees are used as hyperparameters to improve the variance. GBT for classification works on the same principles, but deals with log-likelihood maximization instead of residual minimization.

Categorical features were one-hot encoded, whereby they were first indexed and then converted to n dimensional vectors (for a category consisting of n levels). 

We experimented with standardization techniques using StandardScaler and RobustScaler, but found they did not really help much. This may be because use of feature scaling in the algorithms we chose (LR and GBT) were already robust to features lying in different ranges.

We chose 2015-2018 as our training dataset, and the 2019 data for testing. We considered large periods of time to ensure that the inferred model is able to predict delays due to almost every condition except for rare events not captured in training data.

Since the flights with DEP_DEL15 flag of 1 were only ~18% of the dataset, there was considerable data imbalance, and hence there was a great chance of getting the minor class predictions incorrect (due to the low training examples for that class). To overcome this, we utilized class weights option in the logistic regression, but saw only a very small improvement (suggesting that the variance in our models was very small already). We looked to implement SMOTE, but found it was not readily available in PySpark.

We used GridSearch for hyperparameter tuning, and implemented TrainingValidationSplit (TSV) with 0.75/0.25 ratio during initial stages of our model building before switching to CrossValidation (5-fold CV). We found that TSV gave very similar prediction scores as 5-fold CV, with the added advantage of being able to run the code in practical times (~3hrs compared to >10 hrs).

While our goal was to improve precision, we chose areaUnderPR as our metric of choice for optimization of the tradeoff between precision and recall. One can get a Precision of perfect 1.0 by predicting 0 for all examples! Clearly, this is undesired. For an imbalanced dataset as in our case, an F-beta score may be more appropriate as it provides a configurable mixture of precision vs recall . Specifically, F0.5 score with more emphasis on precision would be a metric of choice if our business requirement was to minimize false positives. However, for practical use, F1 score with balance weightage given to precision and recall may be a healthy compromise, and is one of the key scores that we rely on. 

https://machinelearningmastery.com/fbeta-measure-for-machine-learning/#:~:text=The%20choice%20of%20the%20beta,measure%20or%20the%20F1%2Dscore.

#### * Split Gold Data into Train-Validate, Test

In [192]:
# drop columns have do not have multiple values to avoid erroring out while indexing
if DATASETS == 'Toy':
  gold_df = gold_df.drop("YEAR").drop("QUARTER") 
elif DATASETS == '2015':
  gold_df = gold_df.drop("YEAR")

gold_df = gold_df.na.drop() 

if DATASETS == '2015-2019':
  train = gold_df.filter((gold_df.YEAR== '2015') | (gold_df.YEAR== '2016') | (gold_df.YEAR== '2017')| (gold_df.YEAR== '2018'))
  test = gold_df.filter((gold_df.YEAR== '2019'))
else:
  train, test = gold_df.randomSplit([.80, 0.20], seed=104)
  train.createOrReplaceTempView("train")
  test.createOrReplaceTempView("test")

In [193]:
if VERBOSE:
  display(gold_df.describe().cache())

#### *Balance Data: Generate Class Weights

- Used class weights for Logistic Regression

In [195]:
# Uncomment to generate class weights
# dataset_size=float(train.count())
# numPositives=train.select("DEP_DEL15").where('DEP_DEL15 == 1').count()
# per_ones=(float(numPositives)/float(dataset_size))*100
# numNegatives=float(dataset_size-numPositives)
# print('The number of ones are {}'.format(numPositives))
# print('Percentage of ones are {}'.format(per_ones))
# BalancingRatio= numNegatives/dataset_size
# print('BalancingRatio = {}'.format(BalancingRatio))

# pre-calculated values here to save time
if DATASETS == "Toy":
  BalancingRatio = 0.7517532554108707
elif DATASETS == "2015":
  BalancingRatio = 0.7736956152970962
elif DATASETS == "2015_2017":
  BalancingRatio = 0.7794535029323947
else:
  BalancingRatio = 0.7792115713310707  
train=train.withColumn("classWeights", F.when(train.DEP_DEL15 == 1,BalancingRatio).otherwise(1-BalancingRatio))
train.select("classWeights").show(5)

+-------------------+
 classWeights|
+-------------------+
0.22078842866892934|
0.22078842866892934|
 0.7792115713310707|
 0.7792115713310707|
0.22078842866892934|
+-------------------+
only showing top 5 rows

In [196]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import RobustScaler
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression

"""
# RFormula treats strings as categorical features and one-hot encodes them
# RFormula creates two columns "label" and "features"
# "features" column is a big long string of all features
"""

# add in QUARTER, YEAR if 2015 or larger datasets, resp.
rForm_Toy = (RFormula(formula = "DEP_DEL15 ~ DEST + ORIGIN + MONTH + DAY_OF_MONTH + DAY_OF_WEEK + DISTANCE + DISTANCE_GROUP + CRS_DEP_HOUR + wind + windangle + cig + vis + tmp + dew + LATITUDE + LONGITUDE + ELEVATION + count_CRS_DEP_TIME  + prior_count_DEP_TIME + prior_agg_DEP_DEL15 + prior_avg_DEP_DELAY + prior_avg_TAXI_OUT + prior_avg_CARRIER_DELAY + prior_avg_WEATHER_DELAY + prior_avg_NAS_DELAY + prior_avg_SECURITY_DELAY + prior_avg_LATE_AIRCRAFT_DELAY + holiday + miss_windAngle + miss_wnd + miss_tmp + miss_dew + AIR_TIME_FLAG", featuresCol="num_ohe_features", labelCol="label").setHandleInvalid("skip"))

rForm_2015 = (RFormula(formula = "DEP_DEL15 ~ QUARTER + DEST + ORIGIN + MONTH + DAY_OF_MONTH + DAY_OF_WEEK + DISTANCE + DISTANCE_GROUP + CRS_DEP_HOUR + wind + windangle + cig + vis + tmp + dew + LATITUDE + LONGITUDE + ELEVATION + count_CRS_DEP_TIME  + prior_count_DEP_TIME + prior_agg_DEP_DEL15 + prior_avg_DEP_DELAY + prior_avg_TAXI_OUT + prior_avg_CARRIER_DELAY + prior_avg_WEATHER_DELAY + prior_avg_NAS_DELAY + prior_avg_SECURITY_DELAY + prior_avg_LATE_AIRCRAFT_DELAY + holiday + miss_windAngle + miss_wnd + miss_tmp + miss_dew + AIR_TIME_FLAG", featuresCol="num_ohe_features", labelCol="label").setHandleInvalid("skip"))

rForm = (RFormula(formula = "DEP_DEL15 ~ YEAR + QUARTER + DEST + ORIGIN + MONTH + DAY_OF_MONTH + DAY_OF_WEEK + DISTANCE + DISTANCE_GROUP + CRS_DEP_HOUR + wind + windangle + cig + vis + tmp + dew + LATITUDE + LONGITUDE + ELEVATION + count_CRS_DEP_TIME  + prior_count_DEP_TIME + prior_agg_DEP_DEL15 + prior_avg_DEP_DELAY + prior_avg_TAXI_OUT + prior_avg_CARRIER_DELAY + prior_avg_WEATHER_DELAY + prior_avg_NAS_DELAY + prior_avg_SECURITY_DELAY + prior_avg_LATE_AIRCRAFT_DELAY + holiday + miss_windAngle + miss_wnd + miss_tmp + miss_dew + AIR_TIME_FLAG", featuresCol="num_ohe_features", labelCol="label").setHandleInvalid("skip"))  
  
if DATASETS == "Toy":
  num_ohe_stage = rForm_Toy
elif DATASETS == '2015':
  num_ohe_stage = rForm_2015
else:
  num_ohe_stage = rForm


## Model Implementations

In [198]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

In [199]:
# Helper functions to save the "best model" to disk
gold_data_path = f"/team7/{user_initials}/{DATASETS}/gold_dataset"

def persist_model(tvsFitted, model_name):
  model_path = f"/team7/MR/{DATASETS}/{model_name}"
  best_model = tvsFitted.bestModel
  best_model.write().overwrite().save(model_path)

# load model from disk
def load_model(model_name):
  model_path = f"/team7/MR/{DATASETS}/{model_name}"
  loaded_model = PipelineModel.load(model_path)
  return loaded_model

###Logistic Regression Pipeline

In [201]:
# logistic regression takes in "label" and "features"
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features").setWeightCol("classWeights")
lr_pipeline = Pipeline(stages=[num_ohe_stage, VectorAssembler(inputCols=["num_ohe_features"], outputCol="features"), lr])
lr_params = (ParamGridBuilder().addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
                            .addGrid(lr.regParam, [0.1, 2.0])
                            .build())


### Gradient Boosted Trees (GBT) Pipeline

In [203]:
gbt = GBTClassifier().setLabelCol("label").setFeaturesCol("features")
gbt_pipeline = Pipeline(stages=[num_ohe_stage, VectorAssembler(inputCols=["num_ohe_features"], outputCol="features"), gbt])
gbt_params = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 30])
             .addGrid(gbt.maxIter, [10, 15])
             .build())

In [204]:
def run_and_persist_best_model(model_name, params, pipeline, data):
  evaluator = (BinaryClassificationEvaluator().setMetricName("areaUnderPR")
                                            .setRawPredictionCol("rawPrediction")
                                            .setLabelCol("label"))
  crossval = (CrossValidator(estimator=pipeline,
                          estimatorParamMaps=params,
                          evaluator=evaluator,
                          numFolds=5))  

  # train the model
  crossvalFitted = crossval.fit(data)
  persist_model(crossvalFitted, model_name)

In [205]:
if PERSIST_MODELS:
  run_and_persist_best_model("lr", lr_params, lr_pipeline, train)

In [206]:
if PERSIST_MODELS:
  run_and_persist_best_model("gbt", gbt_params, gbt_pipeline, train)

### Logistic Regression Results

In [208]:
# # evaluate the model
lr_transform_test = load_model("lr").transform(test)

areaUnderROC = BinaryClassificationEvaluator().setMetricName("areaUnderROC").evaluate(lr_transform_test)
areaUnderPR = BinaryClassificationEvaluator().setMetricName("areaUnderPR").evaluate(lr_transform_test)
f1 = MulticlassClassificationEvaluator().setMetricName("f1").evaluate(lr_transform_test)
weightedPrecision = MulticlassClassificationEvaluator().setMetricName("weightedPrecision").evaluate(lr_transform_test)
weightedRecall = MulticlassClassificationEvaluator().setMetricName("weightedRecall").evaluate(lr_transform_test)
accuracy = MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(lr_transform_test)

print("areaUnderROC = ", areaUnderROC)
print("areaUnderPR = ", areaUnderPR)
print("f1 = ", f1)
print("weightedPrecision = ", weightedPrecision)
print("weightedRecall = ", weightedRecall)
print("accuracy = ", accuracy)


areaUnderROC = 0.678005656860986
areaUnderPR = 0.3962256283309613
f1 = 0.678342703717906
weightedPrecision = 0.737480681193688
weightedRecall = 0.6514884789007018
accuracy = 0.6514884789007018

In [209]:
metrics = lr_transform_test.select("label", "prediction").rdd.map(tuple)
metrics = MulticlassMetrics(metrics)
print("Confusion Matrix:", metrics.confusionMatrix().toArray())

Confusion Matrix: [[2031753. 354330.]
 [1003374. 506265.]]

In [210]:
y_true = lr_transform_test.select("label").collect()
y_pred = lr_transform_test.select("prediction").collect()
print(classification_report(y_true, y_pred))

precision recall f1-score support

 0.0 0.85 0.67 0.75 3035127
 1.0 0.34 0.59 0.43 860595

 accuracy 0.65 3895722
 macro avg 0.59 0.63 0.59 3895722
weighted avg 0.74 0.65 0.68 3895722

### GBT Results

In [212]:
# # evaluate the model
gbt_transform_test = load_model("gbt").transform(test)

areaUnderROC = BinaryClassificationEvaluator().setMetricName("areaUnderROC").evaluate(gbt_transform_test)
areaUnderPR = BinaryClassificationEvaluator().setMetricName("areaUnderPR").evaluate(gbt_transform_test)
f1 = MulticlassClassificationEvaluator().setMetricName("f1").evaluate(gbt_transform_test)
weightedPrecision = MulticlassClassificationEvaluator().setMetricName("weightedPrecision").evaluate(gbt_transform_test)
weightedRecall = MulticlassClassificationEvaluator().setMetricName("weightedRecall").evaluate(gbt_transform_test)
accuracy = MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(gbt_transform_test)

print("areaUnderROC = ", areaUnderROC)
print("areaUnderPR = ", areaUnderPR)
print("f1 = ", f1)
print("weightedPrecision = ", weightedPrecision)
print("weightedRecall = ", weightedRecall)
print("accuracy = ", accuracy)


areaUnderROC = 0.6811002316022879
areaUnderPR = 0.4070982086444518
f1 = 0.7237228937142821
weightedPrecision = 0.7594576887089426
weightedRecall = 0.7887872902635249
accuracy = 0.788787290263525

In [213]:
metrics = gbt_transform_test.select("label", "prediction").rdd.map(tuple)
metrics = MulticlassMetrics(metrics)
print("Confusion Matrix:", metrics.confusionMatrix().toArray())

Confusion Matrix: [[2984656. 772355.]
 [ 50471. 88240.]]

In [214]:
y_true = gbt_transform_test.select("label").collect()
y_pred = gbt_transform_test.select("prediction").collect()
print(classification_report(y_true, y_pred))

precision recall f1-score support

 0.0 0.79 0.98 0.88 3035127
 1.0 0.64 0.10 0.18 860595

 accuracy 0.79 3895722
 macro avg 0.72 0.54 0.53 3895722
weighted avg 0.76 0.79 0.72 3895722

## Stacking

In [216]:
gbt_transform_train = load_model("gbt").transform(train)
lr_transform_train = load_model("lr").transform(train)
gbt_transform_train = gbt_transform_train.withColumnRenamed("prediction", "gbt_prediction").withColumn("gbt_prediction", col("gbt_prediction").cast("string"))
lr_transform_train = lr_transform_train.withColumnRenamed("prediction", "lr_prediction").withColumn("lr_prediction", col("lr_prediction").cast("string"))
gbt_transform_train.createOrReplaceTempView("gbt_transform_train")
lr_transform_train.createOrReplaceTempView("lr_transform_train")

In [217]:
stacked_train_predictions_df = lr_transform_train.select('ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR','lr_prediction').join(gbt_transform_train.select('ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR', 'gbt_prediction'), how="left", on=['ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR']).select("DEP_DEL15", "lr_prediction", "gbt_prediction")

stacked_train_predictions_df = (stacked_train_predictions_df.withColumn("DEP_DEL15", col("DEP_DEL15").cast("string")))
stacked_train_predictions_df.createOrReplaceTempView("stacked_train_predictions_df")

In [218]:
stacked_train_predictions_df.printSchema()

root
-- DEP_DEL15: string (nullable = true)
-- lr_prediction: string (nullable = false)
-- gbt_prediction: string (nullable = true)

#### Build Stacking Pipeline

In [220]:
stacked_rForm = (RFormula(formula = "DEP_DEL15 ~ lr_prediction + gbt_prediction").setHandleInvalid("skip"))
meta_gbt = GBTClassifier()#.setLabelCol("label").setFeaturesCol("features")
stacked_pipeline = Pipeline(stages=[stacked_rForm, meta_gbt])

meta_params = (ParamGridBuilder()
             .addGrid(meta_gbt.maxDepth, [2, 4, 6])
             .addGrid(meta_gbt.maxBins, [20, 30])
             .addGrid(meta_gbt.maxIter, [10, 15])
             .build())

if PERSIST_MODELS: 
  run_and_persist_best_model("stacked_best_model", meta_params, stacked_pipeline, stacked_train_predictions_df)

### Evaluation of Test Dataset using Stacking Estimator

In [222]:
# generate lr_prediction and gbt_prediction for test dataset using lr_best_model and gbt_best_model

gbt_transform_test = load_model("gbt").transform(test)
lr_transform_test = load_model("lr").transform(test)
gbt_transform_test = gbt_transform_test.withColumnRenamed("prediction", "gbt_prediction").withColumn("gbt_prediction", col("gbt_prediction").cast("string"))
lr_transform_test = lr_transform_test.withColumnRenamed("prediction", "lr_prediction").withColumn("lr_prediction", col("lr_prediction").cast("string"))
gbt_transform_test.createOrReplaceTempView("gbt_transform_test")
lr_transform_test.createOrReplaceTempView("lr_transform_test")

# create new_test_df with lr_test_prediction and gbt_test_prediction
stacked_test_predictions_df = lr_transform_test.select('ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR','lr_prediction').join(gbt_transform_test.select('ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR', 'gbt_prediction'), how="left", on=['ORIGIN', 'DEST', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'TAIL_NUM', 'DEP_DEL15', 'FLIGHTS', 'CRS_DEP_HOUR']).select("DEP_DEL15", "lr_prediction", "gbt_prediction")

stacked_test_predictions_df = stacked_test_predictions_df.withColumn("DEP_DEL15", col("DEP_DEL15").cast("string"))#.withColumnRenamed("DEP_DEL15", "label")
stacked_test_predictions_df.createOrReplaceTempView("stacked_test_predictions_df")

In [223]:
# load the model and transform the stacked_test_predictions_df
model_name = "stacked_best_model"
loaded_prediction_and_labels = load_model(model_name).transform(stacked_test_predictions_df)

areaUnderROC = BinaryClassificationEvaluator().setMetricName("areaUnderROC").evaluate(loaded_prediction_and_labels)
areaUnderPR = BinaryClassificationEvaluator().setMetricName("areaUnderPR").evaluate(loaded_prediction_and_labels)
f1 = MulticlassClassificationEvaluator().setMetricName("f1").evaluate(loaded_prediction_and_labels)
weightedPrecision = MulticlassClassificationEvaluator().setMetricName("weightedPrecision").evaluate(loaded_prediction_and_labels)
weightedRecall = MulticlassClassificationEvaluator().setMetricName("weightedRecall").evaluate(loaded_prediction_and_labels)
accuracy = MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(loaded_prediction_and_labels)

print("areaUnderROC = ", areaUnderROC)
print("areaUnderPR = ", areaUnderPR)
print("f1 = ", f1)
print("weightedPrecision = ", weightedPrecision)
print("weightedRecall = ", weightedRecall)
print("accuracy = ", accuracy)

areaUnderROC = 0.6409574093192305
areaUnderPR = 0.41581920691593866
f1 = 0.7236853215821056
weightedPrecision = 0.7596274970853623
weightedRecall = 0.7888157830563886
accuracy = 0.7888157830563886

In [224]:
metrics = loaded_prediction_and_labels.select("label", "prediction").rdd.map(tuple)
metrics = MulticlassMetrics(metrics)
print("Confusion Matrix:", metrics.confusionMatrix().toArray())

Confusion Matrix: [[2984934. 772522.]
 [ 50193. 88073.]]

In [225]:
y_true = loaded_prediction_and_labels.select("label").collect()
y_pred = loaded_prediction_and_labels.select("prediction").collect()
print(classification_report(y_true, y_pred))

precision recall f1-score support

 0.0 0.79 0.98 0.88 3035127
 1.0 0.64 0.10 0.18 860595

 accuracy 0.79 3895722
 macro avg 0.72 0.54 0.53 3895722
weighted avg 0.76 0.79 0.72 3895722

[Slide16-Madhukar]

##Model Results

- 5-fold **CrossValidation** did not show any significant improvement over **TrainValidationSplit** in 3:1 ratio. Summary of both are shown below.
>- Training time of CV was signficantly longer, understandably
- **weightedPrecision** and **F1** scores considered for model performance evaluation
- **weightedPrecision improved from LR-->GBT-->Stacking**
- **F1 improved from LR-->GBT** (stacking did not show further improvement)

In [229]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/cross-validation%20results%20on%202015.png?token=AM6SFIRHQYAS4SXEZGLGGMK7GV6S6' >")

In [230]:
#CV results on 2019 (LR and GBT)
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/2019%20v3.png?token=AM6SFIVX2WZLEQOPWLHB52C7GYNWW' >")

In [231]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/TSV%20results%20on%202019.png?token=AM6SFITEXSOVHGMM5YUB4XK7GV6ES' >")

#### Confusion Matrix

In [233]:
displayHTML("<img src ='https://raw.githubusercontent.com/UCB-w261/su20-project-su20-team7/master/images/CM%203.png?token=AM6SFIRVVQUEC6BKRIO2ODK7GV6QG' >")

[Slide17-Max/Rad]

###Conclusions

**Join Performance and Scalability**

The biggest join is between airlines (flights) and weather. We prepared for this by assigning each weather report to a time bucket and making sure to consider only the latest report in each bucket if there were multiple. Pyspark window functions were great for this. Window functions are very expressive and flexible and quite difficult to understand and test. 

The scalability of this join was really not a problem, even with the largest datasets. However, correctness was difficult. We benefited by putting some strict assertions in our code to make sure that we had not dropped any data or inadvertently duplicated it.

**Cloud Computing**

Besides providing the necessary computing resources for our experiments, the cloud makes it possible for the proposed solution to be easily scalable. In fact, if the amount of data increases (e.g., by extending the analysis to many years of flight and weather data), the cloud can provide the required resources with a high level of elasticity, reliability, and scalability.

**Solution Approach**

The following are the overall **merits** of our approach:

 - Many models do not account for weather, but we have a chance to now include weather, which has the potential to improve accuracy of predictions
 - any models predict aggregate delays at major airports which are not specific enough for most stakeholders involved to take action. Delay predictions of individual flights may fill this gap.
 - The pre-vision horizon for most models is 24 hrs, using weather, this can be upto 4-5 days in advance
 - However we did note that most of the reasons for delays are stochastic phenomena which are difficult to predict timely and accurately.

Our **future work** could include:

 - Data balancing using sophisticated techniques such as SMOTE
 - Multi-class classification based on delay duration group - we imagine that in addition to knowing that a flight would be delayed, it would perhaps be more helpful to know whether a flight would be delayed by 15-30 minutes vs 30-45min.
 - Considering multiple time windows for weather related features for greater precision
 - Implementing some sort of page rank for the airports to reflect its capacity to handle delays

### Application of Course Concepts 

**Scalability**: 
One of the first issues we confronted in this project is the sheer size of the dataset: over 300 million records of weather data and gigabytes of flight data. And while we were able to conduct our initial EDA and algorithm exploration on a much smaller subset, we needed to ensure every aspect of our approach was scalable to the entire dataset. We relied on the use of PySpark data structures such as DataFrames as well as keeping all our transformations and other operations on the Spark cluster of machines. This allowed us to scale from or first attempts in this project on the smaller dataset up to the complete version. Only our approach to EDA required a subtle change as we no longer run EDA on the entire dataset but instead on a 1% sample.

**One Hot Encoding / Vector Embeddings**: 
We wanted to train our LG model with certain categorical features such as Origin and Destination airports, an important determinant of departure delays as we showed in our EDA and feature selection sections. We employed helper functions VectorAssembler and RFormula provided by the ML Feature library in order to convert these columns from their original string data types into a series of columns that are one hot encoded to represent their respective airports. This transformation helps binarize the feature vector and make the model fit more efficient.

**Model Complexity / bias variance tradeoff / regularization**: 
If our model is too simple, then we would run the risk of underfitting with high bias. On the other hand, if the model is too complex with many features, then the number of features after OHE etc would explode leading to overfitting with high variance. This is bias-variance tradeoff, and can be partially averted by using ensemble methods like Random Forests, Gradient Boosted Trees etc. For non-ensemble algorithms like logistic regression, L1 and L2 regularization can be utilized. Very high L1 regularization would drive the weights to zero, and thus help to indirectly do feature selection. In our case, we implemented ElasticNet so we could tune the extent of L1 and L2 regularization.

### Research Citations
- The Economic Cost of Airline Flight Delay, Everett B. Peterson, Kevin Neels, Nathan Barczi and Thea Graham,: Journal of Transport Economics and Policy , January 2013, Vol. 47, No. 1 (January 2013), pp. 107-121

- J. J. Rebollo and H. Balakrishnan. Characterization and prediction of air trac delays.Transportation Research Part C: Emerging Technologies, 44(Supplement C):231–241,July 2014

- Using Scalable Data Mining for Predicting Flight Delays, L. Belcastro, Domenico Talia, Fabrizio Marozzo, Paolo Trunfio, ACM Transactions on Intelligent Systems and Technology 8(1) · January 2016

- A Review on Flight Delay Prediction, Alice Sternberg, Jorge Soares, Diego Carvalho, Eduardo Ogasawara CEFET/RJ, Rio de Janeiro, Brazil, November 6, 2017

- Bad Weather and Flight Delays: The Impact of Sudden and Slow Onset Weather Events, Stefan Borsky, Christian Unterberger, Economics of Transportation 18(June):10-26 · March 2019

- A Methodology for Predicting Aggregate Flight Departure Delays in Airports Based on Supervised Learning, Bojia Ye, Bo Liu, Yong Tian, Lili Wan, MDPI Basel, Switzerland, April 2020